# Requisitos previos

In [ ]:
#!pip install gradio
#!pip install neattext
#!pip install emoji
#!pip install scikit-learn==1.4.2 

#!pip install joblib
#!pip install nltk

In [1]:
import platform
import joblib
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
# from googletrans import Translator

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agust\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\agust\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\agust\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\agust\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import gradio as gr

C:\Users\agust\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Funcion axuiliar para saber si estoy en el collab y usar su path o el del proyecto de github
def is_running_on_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False
    
from enum import Enum
# Funcion auxiliar para luego entrenar varios modelos con una sola ejecucion
class Modelos(Enum):
    LOGISTIC_REGRESSION = 'logistic_regression'
    DECISION_TREE = 'decision_tree'
    MULTINOMIAL = 'multinomial'
    BERNOULLI = 'bernoulli'
    GAUSIAN = 'gausian'

# Funcion procesador texto

In [4]:
import neattext.functions as nfx
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import emoji

wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n'


stopwords_en = stopwords.words('english')
stopwords_en = set(stopwords_en).union(set(punctuation))

my_custom_stopwords = {'’', "n't", "'m", "'s", "'ve", '...', 'ca', "''", '``', '\u200d', 'im', 'na', "'ll", '..', 'u', "'re", "'d", '--', '”', '“', '\u200f\u200f\u200e', '....', 'ㅤ','\u200e\u200f\u200f\u200e', 'x200b', 'ive', '.-', '\u200e', '‘'}

stopwords_en = stopwords_en.union(my_custom_stopwords)


def preprocessing_function(text):
    words = []

    for word, tag in pos_tag(word_tokenize(nfx.clean_text(text))):
        word_lemmatized = wnl.lemmatize(word.lower(), pos=penn2morphy(tag))

        if '\u200b' in word_lemmatized:
            continue

        if word_lemmatized not in stopwords_en and not word_lemmatized.isdigit() and not emoji.purely_emoji(word_lemmatized):
            words.append(word_lemmatized)

    return words

# Carga modelo entrenado SUICIDIO

In [5]:
# MODIFICAR ESTOS PARAMETROS PARA LA CARGA
# --------------------------------------------------------
nombre_modelo_prev_entrenado = Modelos.LOGISTIC_REGRESSION.value
# usar formato '25k' para 25.000 filas ejemplo
cant_prev_entrenada = '50k'

path_base_modelo_generado = '/content/' if is_running_on_colab() else ('../tentativa_suicidio/entrenados/' if platform.system() == 'Linux' else '..\\tentativa_suicidio\\entrenados\\')
path_modelo_generado = path_base_modelo_generado + nombre_modelo_prev_entrenado + '_' + cant_prev_entrenada
# --------------------------------------------------------

suicide_model = joblib.load(path_modelo_generado + '_model.pkl')
suicide_vect = joblib.load(path_modelo_generado + '_vector.pkl')

print(type(suicide_model))
print(type(suicide_vect))

<class 'sklearn.linear_model._logistic.LogisticRegression'>
<class 'sklearn.feature_extraction.text.CountVectorizer'>


# Carga modelo entrenado VIOLENCIA

In [6]:
# MODIFICAR ESTOS PARAMETROS PARA LA CARGA
# --------------------------------------------------------
nombre_modelo_prev_entrenado = Modelos.LOGISTIC_REGRESSION.value
# usar formato '25k' para 25.000 filas ejemplo
cant_prev_entrenada = '50k'

path_base_modelo_generado = '/content/' if is_running_on_colab() else ('../violencia/entrenados/' if platform.system() == 'Linux' else '..\\violencia\\entrenados\\')
path_modelo_generado = path_base_modelo_generado + nombre_modelo_prev_entrenado + '_' + cant_prev_entrenada
# --------------------------------------------------------

violence_model = joblib.load(path_modelo_generado + '_model.pkl')
violence_vect = joblib.load(path_modelo_generado + '_vector.pkl')

print(type(violence_model))
print(type(violence_vect))

<class 'sklearn.linear_model._logistic.LogisticRegression'>
<class 'sklearn.feature_extraction.text.CountVectorizer'>


# Funcion predict SUICIDIO

In [7]:
# translator = Translator()

def get_tentativa_suicidio(text_input, english_text=False):
    texto_a_analizar = text_input #if english_text else translator.translate(text_input, dest='en').text
        
    texto_preprocesado = ' '.join(preprocessing_function(texto_a_analizar))
    texto_vectorizado = suicide_vect.transform([texto_preprocesado])

    return bool(suicide_model.predict(texto_vectorizado)[0])

# Funcion predict VIOLENCIA

In [8]:
# translator = Translator()

def get_texto_con_violencia(text_input, english_text=False):
    texto_a_analizar = text_input #if english_text else translator.translate(text_input, dest='en').text
        
    texto_preprocesado = ' '.join(preprocessing_function(texto_a_analizar))
    texto_vectorizado = violence_vect.transform([texto_preprocesado])

    return bool(violence_model.predict(texto_vectorizado)[0])

# Funcion para traduccion

In [13]:
from deep_translator import GoogleTranslator

def translatedSentence(text):
    return GoogleTranslator(source='auto', target='en').translate(text=text)

# Interfaz

In [1]:
def get_answer(message, history):
    message_eng=translatedSentence(message)
    tentativa_suicidio = get_tentativa_suicidio(message_eng, False)
    violento = get_texto_con_violencia(message_eng, False)

    rta_suicida = ("" if tentativa_suicidio else "no" )+ " posee ideas suicidas"
    rta_violento = ("" if violento else "no") + " posee ideas violentas"

    return "El texto ingresado " + rta_suicida + " y " + rta_violento

gr.ChatInterface(
    get_answer,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Por favor ingrese un texto para analizarlo", container=False, scale=7),
    title="Deteccion suicidio y violencia en textos",
    description="Universidad Nacional de la Matanza - Grupo 2 - IA Aplicada - 1c2024",
    theme="soft",
    examples=["I want to jump from a bridge",
                "I want to suicide me",
                 "I hate my parents with all my heart",
                 "I hate all about this life",
                 "I cry every night",
                 "I don't know what is happen to me, but I don't want live anymore",
                 "Nose que me esta pasando, pero ya no quiero vivir mas",
                 "Non so cosa mi sta succedendo, ma non voglio più vivere."],
    cache_examples=False,
    retry_btn=None,
    undo_btn="Limpiar todo",
    clear_btn="Limpiar Historial Charla",
).launch()


NameError: name 'gr' is not defined